# Experiments with the Conv-VAE code.

In [5]:
import sys
sys.path.append("..")
import settings

CONV_VAE_PATH = "/home/lboloni/Documents/Hackingwork/_Checkouts/Julian-8897-Conv-VAE-PyTorch/Conv-VAE-PyTorch"
sys.path.append(CONV_VAE_PATH)

In [6]:
import argparse
import numpy as np
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from torch.nn import functional as F
import torchvision.utils as vutils
from torchvision import transforms
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import argparse

from mpl_toolkits.axes_grid1 import ImageGrid
#Fixes PosixPath Error
import pathlib

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

## Loading a pretrained code

This is basically taken from the test.py

# Experiment to load the model and print it out

This main function takes a config object. It is not clear for me what type it is, but I think that it is the ConfigParser from parse_config. 

The problem is that that file is initialized exclusively from the command line. How do I put it in a notebook?

In [10]:
# As the code is highly dependent on the command line, emulating it here
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str,
                     help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                      help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str,
                      help='indices of GPUs to enable (default: all)')
# value = ["-cconfig.json", "-rsaved/models/VAE_CelebA/0817_120220/checkpoint-epoch20.pth"]
# value = ["-cconfig-robot.json", "-rsaved-robot/models/VAE_Robot/0817_155635/checkpoint-epoch20.pth"]

# FIXME: make this dependent of the machine
JSON_PATH = "config-robot-tredy2.json"
ROBOT_MODEL_PATH = "/home/lboloni/Documents/Hackingwork/__Temporary/VisionBasedRobotManipulator-models/Conv-VAE/models/VAE_Robot/0817_155635/checkpoint-epoch20.pth"

value = ["this-script", f"-c{JSON_PATH}", f"-r{ROBOT_MODEL_PATH}"]

# we are changing the parameters from here, to avoid changing the github 
# downloaded package
sys.argv = value
config = ConfigParser.from_args(args)
print(config)

In [11]:
logger = config.get_logger('test')

# setup data_loader instances
# LOTZI: this is a specific data loader, which with the current implementation, it is very specific to the celeba. It needs to be changed...

# FIXME: Initially I changed this data loader to something else and added into the data_loader a new one called RobotDataLoader. 
# But I should not change the github downloaded package. 
# So for the time being, I changed it back to CelebDataLoader, with the 
# awareness that I can specify something in config and update it correspondingly here 

data_loader = getattr(module_data, config['data_loader']['type'])(
    config['data_loader']['args']['data_dir'],
    batch_size=36,
    shuffle=False,
    validation_split=0.0,
    # training=False,
    num_workers=2
)

# LOTZI: this is an uninitialized model architecture
# build model architecture
model = config.init_obj('arch', module_arch)
logger.info(model)

# get function handles of loss and metrics
loss_fn = getattr(module_loss, config['loss'])
# metric_fns = [getattr(module_metric, met) for met in config['metrics']]



INFO:test:VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
 

In [12]:
# logger.info('Loading checkpoint: {} ...'.format(config.resume))
# checkpoint = torch.load(config.resume)

# loading on CPU-only machine
print("Loading the checkpoint")
checkpoint = torch.load(config.resume, map_location=torch.device('cpu'))
print("Checkpoint loaded")

state_dict = checkpoint['state_dict']
if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()


Loading the checkpoint
Checkpoint loaded


/tmp/ipykernel_102589/2087784403.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(config.resume, map_location=torch.device('cpu'))


VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Se

In [13]:
print(model)
print(model.encoder)
print(f"latent_dim {model.latent_dim}")
# print(model.hidden_dims)

VanillaVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Se

## Pass in one picture

In [14]:
from PIL import Image
import torchvision.transforms as transforms

# picture_name = '../../localdata/img/Rafael.jpg'
# picture_name = '../../localdata/img/indian-man.jpeg'
picture_name = '../../localdata/img/00029_dev2.jpg'

# Load an image using PIL
image = Image.open(picture_name)

# Define a transform pipeline (e.g., converting to tensor)
# transform = transforms.ToTensor()
image_size = 64

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
])

# Apply the transform
image_tensor = transform(image)

# Display some information about the image tensor
print(image_tensor.shape)  # e.g., torch.Size([3, H, W])

# Convert the tensor to a format suitable for matplotlib (from [C, H, W] to [H, W, C])
image_tensor_for_pic = image_tensor.permute(1, 2, 0)
plt.imshow(image_tensor_for_pic)


FileNotFoundError: [Errno 2] No such file or directory: '/home/lboloni/Documents/Hackingwork/_Checkouts/VisionBasedRobotManipulator/localdata/img/00029_dev2.jpg'

In [ ]:

# Add a batch dimension: shape becomes [1, 3, 224, 224]
image_tensor_2 = image_tensor.unsqueeze(0)

# Move tensor to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_tensor_3 = image_tensor_2.to(device)

# Running the input on the output
output, mu, logvar = model(image_tensor_3)

# Output: the visual reconstruction
#print(output)
output_for_pic = output[0].cpu().permute(1, 2, 0).detach().numpy()
plt.imshow(output_for_pic)

# mu: the encoding, I think
print(f"mu = {mu}")

# log_var: the log-var values for this input 
print(f"logvar = {logvar}")

In [ ]:
# Try to do a decoding from the same mu
z2 = model.reparameterize(mu, logvar)
z2[0][1] = 2.0
output2 = model.decode(z2)
output_for_pic2 = output2[0].cpu().permute(1, 2, 0).detach().numpy()

# initial and new
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(output_for_pic)
axs[1].imshow(output_for_pic2)
